In [1]:
"""
"""

'\n'

In [51]:
import json
import random
import numpy as np
from sklearn import neighbors
import math
import functools

In [52]:
# 读取标签、训练集以及测试集
with open("categories.json", 'r') as file:
    categories = json.load(file)
with open("train_data_features_2.json", 'r') as file:
    train_data = json.load(file)
with open("test_data_features_2.json", 'r') as file:
    test_data = json.load(file)
with open("weight.json", 'r') as file:
    w = json.load(file)

In [53]:
# 将训练集转化为Sklearn 能识别的数据集，并且声称相对应的标签
train_data_tmp = None
train_label = None
for i, (key, value) in enumerate(train_data.items()):
    value = np.array(value)
    if train_data_tmp is None:
        train_data_tmp = value.copy()
    else:
        train_data_tmp = np.vstack((train_data_tmp, value))

    if train_label is None:
        train_label = np.full((len(value)), i)
    else:
        train_label = np.hstack((train_label, np.full((len(value)), i)))

train_data = train_data_tmp

# 减去判定为影响误差的向量 10/14/18/22
train_data = np.delete(train_data, 22, 1)
train_data = np.delete(train_data, 18, 1)
train_data = np.delete(train_data, 14, 1)
train_data = np.delete(train_data, 10, 1)

print(train_data.shape)
print(train_label.shape)
# print(train_data)

(600, 21)
(600,)


In [54]:
# 将测试集转化为Sklearn 能识别的数据集，并且声称相对应的标签
test_data_tmp = None
test_label = None
for i, (key, value) in enumerate(test_data.items()):
    value = np.array(value)
    if test_data_tmp is None:
        test_data_tmp = value.copy()
    else:
        test_data_tmp = np.vstack((test_data_tmp, value))

    if test_label is None:
        test_label = np.full((len(value)), i)
    else:
        test_label = np.hstack((test_label, np.full((len(value)), i)))

test_data = test_data_tmp

# 减去判定为影响误差的向量 10/14/18/22
test_data = np.delete(test_data, 22, 1)
test_data = np.delete(test_data, 18, 1)
test_data = np.delete(test_data, 14, 1)
test_data = np.delete(test_data, 10, 1)

print(test_data.shape)
print(test_label.shape)
print(test_data)

(400, 21)
(400,)
[[ 0.04871063  0.32012228  0.4445425  ...  0.05255349  0.01872143
   0.13029645]
 [ 0.09616957  0.22175108  0.32533339 ...  0.40603646  0.23073328
   0.39633191]
 [ 0.04261603  0.38468111  0.48362982 ...  0.00190095 -0.00109035
   0.0006649 ]
 ...
 [ 0.46644582  0.          0.         ...  0.0074192   0.01516534
   0.12082561]
 [ 0.18078634  0.4174556   0.36608708 ...  0.06183568  0.04101134
   0.21589503]
 [ 0.55814159  0.49388387  0.1538018  ...  0.06409022  0.12517175
   0.42217396]]


In [55]:
# 剪辑近邻法去除不易区分的点
train_tmp = np.hstack((train_data, train_label.reshape(-1, 1)))
np.random.shuffle(train_tmp)

In [56]:
while True:
    train_test_size = int(len(train_tmp)/2)
    train_test = train_tmp[0:train_test_size, :]
    train_refer = train_tmp[train_test_size:, :]
    knn = neighbors.KNeighborsClassifier()
    knn.fit(train_refer[:, 0:21], train_refer[:, 21])

    train_test_hat = knn.predict(train_test[:, 0:21])
    train_test_corr = (train_test_hat == train_test[:, 21])

    has_change = False
    for i in range(train_test_size-1,-1,-1):
        # print(i)
        if not train_test_corr[i]:
            train_test = np.delete(train_test, i, 0)
            has_change = True
    break

    if not has_change:
        break

    train_tmp = train_test

print(train_test.shape)


(163, 22)


In [57]:
# 重新生成训练数据和标签
train_data, train_label = train_test[:, 0:21], train_test[:, 21]


In [58]:
# 创建KNN分类器， K=5
knn=neighbors.KNeighborsClassifier()
knn.fit(train_data, train_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [64]:
# 获取正确率
print(test_label.shape)
test_label_hat = knn.predict(test_data)
print((test_label_hat == test_label).sum())



(400,)
217


In [65]:
cor_sum=0
for x in correct:
    cor_sum += x
print(cor_sum / 400)


0.0
